# The p-Median Problem

[Tutorial](https://nbviewer.jupyter.org/github/Pyomo/PyomoGallery/blob/master/p_median/p_median.ipynb)

In [1]:
import random
import re

from pyomo import environ as pyo
from pyomo.dataportal import DataPortal
from pyomo.opt import SolverFactory

random.seed(1000)

infinity = float("inf")

## Problem

Locate $p$ facilities to minimize the demand-weighted average distance between demand nodes and the nearest selected facilities.

### Sets

- $M$: set of candidate locations
- $N$: set of customer demand nodes

### Parameters

- $p$: number of facilties to locate
- $d_j$: demand of customer $j$, $\forall j \in N$
- $c_{ij}$: unit cost of satisfying customer $j$ from facility $i$, $\forall i \in M$, $\forall j \in N$

### Variables

- $x_{ij}$: fraction of the demand of customer $j$ that is supplied by facility $i$, $\forall i \in M$, $\forall j \in N$
- $y_i$: a binary value that is $1$ when a facility is located at location $i$, $\forall i \in M$

### Objective

Minimize the demand-weighted total cost: $\min \sum_{i \in M} \sum_{j \in N} d_j c_{ij} x_{ij}$

### Constraints

- satisfy all customer demand: $\sum_{i \in M}  x_{ij} = 1$, $\forall j \in N$
- exactly $p$ facilities are located: $\sum_{i \in M} y_i = p$
- demand nodes can only be assigned to open facilities: $x_{ij} \leq y_i$, $\forall i \in M$, $\forall j \in N$
- assignment variables must be non-negative: $x_{ij} \ge 0$, $\forall i \in M$, $\forall j \in N$

## Coding the problem

In [2]:
model = pyo.AbstractModel()

### Sets

In [3]:
model.m = pyo.Param(within=pyo.PositiveIntegers)
model.n = pyo.Param(within=pyo.PositiveIntegers)
model.M = pyo.RangeSet(1, model.m)
model.N = pyo.RangeSet(1, model.n)

### Parameters

In [4]:
model.p = pyo.Param(within=pyo.RangeSet(1, model.n))
model.d = pyo.Param(model.N, default=1.0)
model.c = pyo.Param(
    model.M,
    model.N,
    initialize=lambda i, j, model: random.uniform(1.0, 2.0),
    within=pyo.Reals,
)

### Variables

In [5]:
model.x = pyo.Var(model.M, model.N, bounds=(0.0, 1.0))
model.y = pyo.Var(model.M, within=pyo.Binary)

### Objective

In [6]:
def cost_(model):
    return sum(
        model.d[j] * model.c[i, j] * model.x[i, j] for i in model.M for j in model.N
    )


model.cost = pyo.Objective(rule=cost_)

### Constraints

In [7]:
def demand_(model, j):
    return sum(model.x[i, j] for i in model.M) == 1.0


def facilities_(model):
    return sum(model.y[i] for i in model.M) == model.p


def openfac_(model, i, j):
    return model.x[i, j] <= model.y[i]


model.demand = pyo.Constraint(model.N, rule=demand_)
model.facilities = pyo.Constraint(rule=facilities_)
model.openfac = pyo.Constraint(model.M, model.N, rule=openfac_)

### Data

In [8]:
data = {None: {"m": {None: 10}, "n": {None: 6}, "p": {None: 3}}}
data

{None: {'m': {None: 10}, 'n': {None: 6}, 'p': {None: 3}}}

### Solving

In [9]:
instance = model.create_instance(data=data, name="The p-Median Problem")
opt = SolverFactory("glpk")
solution = opt.solve(instance)

In [10]:
print(solution)


Problem: 
- Name: unknown
  Lower bound: 6.43118493935767
  Upper bound: 6.43118493935767
  Number of objectives: 1
  Number of constraints: 68
  Number of variables: 71
  Number of nonzeros: 191
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.015077829360961914
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [11]:
for o in instance.component_data_objects(pyo.Objective, active=True):
    print(f"{o}: {pyo.value(o):.5f}")

cost: 6.43118


In [12]:
def print_x_result(v, val):
    ij = v.name.replace("x[", "").replace("]", "").split(",")
    i = int(ij[0])
    j = int(ij[1])
    print(
        f"fraction of the demand of customer {j} that is supplied by facility {i}: {val}"
    )


def print_y_result(v, val):
    j = re.findall("[0-9]+", v.name)[0]
    print(f"facility located at location {j}")


for v in instance.component_data_objects(pyo.Var, active=True):
    value = pyo.value(v)
    if value > 0:
        if "x" in v.name:
            print_x_result(v, value)
        elif "y" in v.name:
            print_y_result(v, value)

fraction of the demand of customer 4 that is supplied by facility 3: 1.0
fraction of the demand of customer 1 that is supplied by facility 6: 1.0
fraction of the demand of customer 2 that is supplied by facility 6: 1.0
fraction of the demand of customer 3 that is supplied by facility 6: 1.0
fraction of the demand of customer 5 that is supplied by facility 6: 1.0
fraction of the demand of customer 6 that is supplied by facility 9: 1.0
facility located at location 3
facility located at location 6
facility located at location 9


---

In [13]:
%load_ext watermark
%watermark -d -u -v -iv -b -h -m

Last updated: 2020-12-21

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.19.0

Compiler    : Clang 10.0.0 
OS          : Darwin
Release     : 20.1.0
Machine     : x86_64
Processor   : i386
CPU cores   : 4
Architecture: 64bit

Hostname: JHCookMac.local

Git branch: master

pyomo: 5.7.2
re   : 2.2.1

